# Introduction


# Data Cleaning

In [2]:
import seaborn as sns #this is the plotting library I'll be using 
import pandas as pd #"as pd" means that we can use the abbreviation in commands
import matplotlib.pyplot as plt #we need Matplotlib for setting the labels in the Seaborn graphs
from statistics import variance
import math
from sklearn.linear_model import LinearRegression #remember, this is linear regression model from sk-learn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

df1 = pd.read_csv('steps.csv', sep = ';')
#df1.head()
#only dates

df2 = pd.read_csv('survey.csv')
#df2.head()
#different columns with ID, city, gender, age, etc.

In [3]:
df = pd.merge(df1, df2, on = 'id') #column of id is the common identifier
#df.head()

In [ ]:
sns.displot(df['weight'].dropna(), kde=False) #Selecting the weight column.
plt.title('Weight of Fitbit Users')
plt.xlabel('Weight in kg')
plt.ylabel('Count')
plt.show()

In [ ]:
def above_200(x): #anything above 200kg in the column "weight"
    if(x > 200): 
        return float('NaN')
    else: 
        return x
df['weight'] = df['weight'].apply(above_200) #apply it to the entire column "weight"

In [ ]:
def below_40(x): #anything above 200kg in the column "weight"
    if(x < 40): 
        return float('NaN')
    else: 
        return x
df['weight'] = df['weight'].apply(below_40) #apply it to the entire column "weight"

In [ ]:
sns.displot(df['weight'].dropna(), kde=False) #Selecting the rating column. I need to drop the NA's for the plot
plt.title('Weight of Fitbit Users')
plt.xlabel('Weight in kg')
plt.ylabel('Count')
plt.show()

#### Create the variable mean steps per participant. Hint: combine Pandas .loc and .mean() with the right axis argument (axis=....) for mean(). Look up the documentation of .loc and Pandas .mean() if you're unsure

In [5]:
df['mean'] = df.iloc[0:,2:329].mean(axis=1) #are the rows correct (2:329)???

#### Show the head() of the resulting data frame


In [19]:
df.head()

,id,20-6-2013,21-6-2013,22-6-2013,23-6-2013,24-6-2013,25-6-2013,26-6-2013,27-6-2013,28-6-2013,...,13-5-2014,city,gender,age,hh_size,education,education_1,height,weight,mean
0,1,NaN,NaN,NaN,NaN,3941.0,15733.0,9929.0,12879.0,10541.0,...,NaN,Bordeaux,Male,25-34,4,4,Master or doctorate,178.0,98.0,10205.521212
1,2,NaN,NaN,10473.0,705.0,4287.0,5507.0,4024.0,3926.0,14595.0,...,NaN,Lille,Male,35-44,1,3,Bachelor,180.0,77.0,5687.423313
2,3,NaN,11428.0,12523.0,2553.0,190.0,2164.0,8185.0,9630.0,8983.0,...,NaN,Montpellier,Male,25-34,2,2,Master or doctorate,180.0,83.0,8301.729730
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Lyon,Male,<25,1,1,Bachelor,178.0,80.0,3633.200000
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Montpellier,Female,25-34,3,4,Bachelor,167.0,61.0,5312.129630


# Exploratory Data Analysis 
### An analysis of mean steps per participant: Graph of the distribution
#### Histogram
According to Healthline, the average steps per day for French people is 5,141. Furthermore, very active jobs - for example, waiters and nurses, "still" only have 22778 steps/day (waiters) & 16390 steps/day (nurses). Hence, the decision was made to clean the "mean" data for steps below 30000 steps/day. 
https://www.healthline.com/health/average-steps-per-day#country

#### mean, median, variance
Variance measures the spread of random data in a set from its mean or median value. A low value for variance indicates that the data are clustered together and are not spread apart widely, whereas a high value would indicate that the data in the given set are much more spread apart from the average value. 

In [ ]:
mean_size = df['mean'].mean()

In [ ]:
median_size = df['mean'].median()

In [ ]:
print("Variance of 'mean' is % s " %(variance(df['mean']))) 

In [ ]:
df = df[(df['mean'] < 30000)]
sns.displot(df['mean'].dropna(), kde=False) #Selecting the rating column. I need to drop the NA's for the plot
plt.title('Average steps per day')
plt.axvline(median_size, 0, 100, color='green', label='median') #This adds a vertical line at x-position median_size, from y = 0 to y = 100 
plt.axvline(mean_size, 0, 100, color='blue', label='mean') 
plt.legend()
plt.xlabel('Steps/Day')
plt.ylabel('Count of Participants')
plt.show()

The mean is 6705 steps/day.
The median is 6240 steps/day.
Variance of 'mean' is 8715971. As visible in the histogram, the data is slightly positively skewed. However, before cleaning the data according to 30000 steps/day, possible outliers were to be identified. 

#### The relation of mean steps per participant with two categorical variables (e.g. gender, age, education)
Graph of the relation (e.g. histograms, box plots, density plots, violin plots). Try a different plot this time.
Describe and interpret the results

# Feature Engineering
The prediction of the model can be improved by computing a variable that gives a better indication of a healthy weight than weight alone. Can you think of one? If not, just skip this step.

BMI: Body mass index (BMI) is a measure of body fat based on height and weight that applies to adult men and women.

BMI = kg/m2 where kg is a person's weight in kilograms and m2 is their height in metres squared.

In [ ]:
df['height in metres'] = df['height'] / 100

In [ ]:
df['BMI'] = df['weight'] / df['height in metres']**2

In [ ]:
sns.displot(df['BMI'].dropna(), kde=False) #I need to drop the NA's for the plot
plt.title('BMI of Fitbit Users')
plt.xlabel('BMI')
plt.ylabel('Count')
plt.show()

# Exploratory Data Analysis (Week 2)
See last week for the first part (distribution of steps). In addition:
Make a scatterplot matrix of the numerical variables
Make a correlation matrix of the numerical variables
Describe and interpret the results

In [6]:
corr = df[['height', 'weight', 'education']].corr() 
print(corr)

             height    weight  education
height     1.000000  0.265138  -0.046343
weight     0.265138  1.000000   0.029968
education -0.046343  0.029968   1.000000


#### Scatterplot Matrix

#### Correlation Matrix

In [ ]:
from scipy.stats.stats import pearsonr #We will need the correlation function from the scientific package scipy
def gender(x):
    if "Female" in x:
        return 1
    else: return 2
    
df['gender Values'] = df['gender'].apply(gender)

In [ ]:
df['gender Values'].dropna()
df['weight'].dropna()

In [ ]:
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns) #the last two arguments set the column names
plt.show()

In [5]:
from scipy.stats.stats import pearsonr #We will need the correlation function from the scientific package scipy
print(pearsonr(df['education'],df['weight']))

ValueError: array must not contain infs or NaNs

Describe and interpret the results

# Predictive Model
## Explain linear regression in your own words
Linear regression is a model that estimates the relationship between one independent variable and one dependent variable using a straight line.
Example: Based on a body weigth of 70kg (dependent variable, y), height should be xx cm (independent variable). 
## Create dummy variables for the categorical variables


In [22]:
pd.get_dummies(df['gender']).head()

,Female,Male
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [21]:
pd.get_dummies(df['education_1']).head()

,Bachelor,High school,Master or doctorate,No degree,Primary,Vocational
0,0,0,1,0,0,0
1,1,0,0,0,0,0
2,0,0,1,0,0,0
3,1,0,0,0,0,0
4,1,0,0,0,0,0


## Fit a linear regression model to predict the mean steps per participant, using the training data. You can include all variables (easy), or you can try to find the combination that gives you the best fit (harder).

In [13]:
#from sklearn.model_selection import train_test_split

In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

ValueError: Found input variables with inconsistent numbers of samples: [927, 925]

In [30]:
print("shape of output - training set", y_train.shape)
print("shape of input - testing set", x_test.shape)
print("shape of output - testing set", y_test.shape)

AttributeError: 'list' object has no attribute 'shape'

In [27]:
from sklearn.linear_model import LinearRegression #we need this specific model from sk-learn

x = df[['weight']].dropna() 
y = df[['mean']].dropna()
lm = LinearRegression() #this creates a new LR model
lm.fit(x, y) #this "fits" the model

b0 = lm.intercept_ #lm.coef_ gets a list of coefficients (excluding the intercept). [0] gets the actual number from the list
b1 = lm.coef_[0] #gets the intercept

print(f"The regression line is equal to y = {b0:.2f} + {b1:.2f}X") #.0f formats to 2 decimals.

ValueError: Found input variables with inconsistent numbers of samples: [927, 925]

In [ ]:
df['mean_p'] = lm.predict(x) #use built-in predict() method to generate predictions; store in original dataframe
df.head(50)

In [ ]:
r2_score(df['mean'], df['mean_p'])

In [ ]:
math.sqrt(mean_squared_error(df['mean'], df['mean_p'])) #this gives the model fit (R2) for the Y and predicted Y

# Evaluation

# Conclusion